# Data Cleaning and Overview of City of Vancouver Animal Control Register dataset
## Objective of this Notebook
The purpose of this notebook is to overview the data contained in the non-administrative columns of the "City of Vancouver Animal Control Register" dataset. This means that the data is focused on the observed traits of animals brought into the care of Vancouver Animal Control.
## Data  Frame Modification Methodology
Where appropriate, the names of categorical entries have been changed for the sake of making the data more understandable.
## Figure Format Methodology
Some columns in this dataset are intended to be categorical, but have so many different categories that they cannot be succinctly displayed in a single figure without doing some kind of grouping. In the case of these columns, I have (somewhat arbitrarily) decided that only the eight most relevant categories will be shown in any figure. Similarly, if there is a categorical variable that only has a few valid categories, but there are many data-entry errors in the dataset, I will only display the valid categories in the figure.
## Imports, Styling, and Loading the Dataset

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Matplotlib styling
plt.style.use('fivethirtyeight')
plt.rcParams["axes.titlesize"] = 30
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["xtick.labelsize"] = 18
plt.rcParams["ytick.labelsize"] = 18

#reading in datafile and set the index to the AnimalID column
df1 = pd.read_csv("../input/city-of-vancouver-animal-control-register/Vancouver-animal-control-inventory-register.csv", sep = ';', header = 'infer')

# Global Data Cleaning

In [ ]:
#convert all of the strings in the datafile to lowercase
df1 = df1.applymap(lambda s:s.lower() if type(s) == str else s)

#remove colums that contain human information, and administrative information as this overview is with inherent qualities and outcomes
df1 = df1.drop(['ACO','ReceiptNumber','PitNumber','ShotsDate','AnimalID','KennelNumber','EnteredBy','Name','DispositionDate','DateImpounded'], axis=1)

# Figures and Observations

In [ ]:
#figure1
#Drop rows where sex = x, and represent sex character codes in a more undertandable way
dfSex = df1.copy()
dfSex = dfSex[(dfSex.Sex != 'x')]
dfSex['Sex'] = dfSex['Sex'].replace({'m':'male non-neutered','m/n':'male neutered','f':'female non-spayed','f/s':'female spayed'})

fig, axarr = plt.subplots(1, 2, gridspec_kw={'width_ratios': [7, 3]},figsize=(21, 5)) 
sns.countplot(x='Sex', data=dfSex, order = dfSex['Sex'].value_counts().index, ax=axarr[0])
axarr[0].set_title("Animal Sex and Neuter/Spay Status")

df1['Age category'].value_counts().plot(kind='pie', ax=axarr[1], )
axarr[1].set_title("Animal Age Catagory")
axarr[1].axes.yaxis.set_visible(False)

### Sex
The majority of the entries (whether neutered or not) are male. It is also worth noting that while the proportion of male and female animals that have not had sterilization surgeries is very similar, there are far more neutered male animals than there are spayed female animals.
### Age
The methodology behind how this data was entered into the dataset is unfortunately unclear, but what can be gathered from the field is that the majority of animals taken in were adults. The "puppy" category is somewhat interesting, because this dataset does not exclusively contain dogs. The category was probably created because most of the entries are dogs however.

In [ ]:
#figure2
figTwo, axarr = plt.subplots(1, 2, figsize=(21, 5))
sns.countplot(x='Source', data=df1, order = df1['Source'].value_counts().iloc[:4].index, ax=axarr[0])
axarr[0].set_title("Animal Source")

sns.countplot(x='Status', data=df1, order = df1['Status'].value_counts().iloc[:4].index, ax=axarr[1])
axarr[1].set_title("Animal Status")
plt.subplots_adjust(hspace=.5)

### Source and Status
There isn't much to observe here other than that most of the animals in the database were captured strays, and that most commonly, animals brought into the care of Vancouver Animal Control, are either picked up by, or delivered to their original owners. It should be noted that while only four categories are shown in the figures for each of these columns, each of these columns have many other (seldom used) categories. 

In [ ]:
#figure3
plt.figure(figsize=(22,6))
sns.countplot(x='Breed', data=df1, order = df1['Breed'].value_counts().iloc[:8].index)
plt.title('Animal Breed')

### Breed
In it's current state, this column may be somewhat valuable for observing which breeds of dogs are especially likely or unlikely to fall into the care of Vancouver Animal Control, but more data cleaning and restructuring is likely needed. In order to make this column more useable it is likely necessary to find decide on how cross-breed animals (denoted with an 'x') should be handled. Th most significant question in this regard is whether purebred animals of a particular breed should be grouped with their mixed equivalent breeds, or if mixed breed animals should be grouped together in their own category.

In [ ]:
#figure4
plt.figure(figsize=(22,6))
sns.countplot(x='Color', data=df1, order = df1['Color'].value_counts().iloc[:8].index)
plt.title('Animal Colour')

### Colour
This column could be more useful with some data restructuring. The best way to do this may just be to group together any animals that have '&' listed in their Colour under a more generic 'mixed colour' category.

In [ ]:
#figure 5
#Detect number of kilogram measurments and drop them (may come back and convert these values to pounds)
dfWeight = df1.copy()
dfWeight = dfWeight.dropna(subset=['ApproxWeight'])
dfWeight = dfWeight[~dfWeight.ApproxWeight.str.contains("kg")]

#removes alphabeticals in Weight column
dfWeight['ApproxWeight'] = dfWeight.ApproxWeight.str.replace(r"[a-zA-Z]",'')

#removes symbols in Weight column including decimals (may come back and remove values after decimal point)
dfWeight['ApproxWeight'] = dfWeight.ApproxWeight.str.replace('\D','')

#convert Weight to numeric, and transform what cannot be converted into NAN, then drop all NANs
dfWeight['ApproxWeight'] = pd.to_numeric(dfWeight['ApproxWeight'], errors='coerce')
dfWeight = dfWeight.dropna(subset=['ApproxWeight'])
dfWeight['ApproxWeight'] = dfWeight['ApproxWeight'].astype('int')

#drop rows with weights over 200 (will fix with proper Outlier detection)
dfWeight = dfWeight[(dfWeight.ApproxWeight < 175)]
dfWeight = dfWeight[(dfWeight.ApproxWeight > 0)]
plt.figure(figsize=(22,8))
sns.kdeplot(data=dfWeight['ApproxWeight'], shade=True)
plt.title('Animal Weight Distribution (Pounds)')

### Weights
The two peaks in the distribution of animal weights may suggest that there is a weight separation between dogs in the dataset and all other domestic animals (which are generally smaller than dogs). It is worth noting that while most of this field had entries in pounds, there were some kilogram entries (which were dropped from this figure).

In [ ]:
#figure 6
#represent walk-ability character codes in a more undertandable way
dfWalkCode = df1.copy()
dfWalkCode['Code'] = dfWalkCode['Code'].replace({'green':'easy','yellow':'moderate','blue':'challenging'})

plt.figure(figsize=(22,6))
sns.countplot(x='Code', data=dfWalkCode, order = dfWalkCode['Code'].value_counts().iloc[:3].index)
plt.title('Animal Walk-Ability Rating')

### Walk-Ability Rating
This variable outlines how easy it is to take the associated animal on a walk, which is presumably determined by either attempting to take the animal on a walk and reporting on the experience, or by making an evaluation based on the animals general behavior. The original data frame has categories encoded mainly as 'green', 'yellow', and 'blue', which represent 'easy', 'moderate', and 'challenging', respectively. There are however a good number of in-between values (ex: 'green/yellow') and incorrectly entered values (ex: 'yel'). It's possible that some of this data could be extracted and recategorized by making comparisons to the valid categories via Levenshtein distance.

# Conclusion
This dataset has some interesting variables that could potentially be used to make behavior predictions (namely walk-ability code), or even make outcome predictions based on the described qualities of the animals brought into Vancouver Animal Control. With that in mind though, this dataset has an issue with data entry quality. Some of this can likely be mitigated with data cleaning and re-engineering, however the original creators of this dataset would benefit from switching from a typed-entry system to a selected-entry field for many of the categorical variables. Similarly, a standard should be established around whether metric or imperial weights are to be used when doing data entry, and entries for the 'ApproxWeight' field should be numeric.  

While there are many rows with NAN values, the dataset is sufficiently large enough to do a regression analysis despite that, and it may be possible to make some predictions with a random forest algorithm, or a neural network, which I will likely attempt in another notebook.